<a href="https://colab.research.google.com/github/nikolasavra/Computational_Intelligence_Project/blob/main/Computational_Intelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [ ]:
import re
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.regularizers import l2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!nvidia-smi

Sun May  8 22:20:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    73W / 149W |  10991MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
#check if GPU is available by Colab
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")

GPU not available


# Uploading data

In [5]:
train_label = pd.read_csv("/content/drive/MyDrive/CI_Project/train-label.csv")
train_label

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,0,1,0,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,1,1,...,0,0,1,0,0,0,0,0,0,0
4,4,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246,8246,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8247,8247,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8248,8248,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8249,8249,1,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


In [6]:
train_data = pd.read_csv("/content/drive/MyDrive/CI_Project/train-data.csv")
train_data

,Unnamed: 0,0,1,10,100,1000,1001,1002,1003,1004,...,990,991,992,993,994,995,996,997,998,999
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246,8246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8247,8247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8248,8248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8249,8249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
test_label = pd.read_csv("/content/drive/MyDrive/CI_Project/test-label.csv")
test_label

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,3978,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3979,3979,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3980,3980,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3981,3981,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
test_data = pd.read_csv("/content/drive/MyDrive/CI_Project/test-data.csv")
test_data

,Unnamed: 0,0,1,10,100,1000,1001,1002,1003,1004,...,990,991,992,993,994,995,996,997,998,999
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,3978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3979,3979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3980,3980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3981,3981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Processing our Data

In [9]:
X = train_data
y = train_label
X_final = test_data
y_final = test_label

In [10]:
X.drop(['Unnamed: 0'], axis=1, inplace=True)
y.drop(['Unnamed: 0'], axis=1, inplace=True)
X_final.drop(['Unnamed: 0'], axis=1, inplace=True)
y_final.drop(['Unnamed: 0'], axis=1, inplace=True)

In [11]:
#Turn dataframes into arrays

X = X.values
y = y.values
X_final = X_final.values
y_final = y_final.values

# Building our Model

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
loss_per_fold = []
acc_per_fold = []
mse_per_fold = []


for i, (train,test) in enumerate(kfold.split(X)):

    model = Sequential()
    model.add(Dense((X.shape[1] + y.shape[1]), activation='relu', input_dim=X.shape[1])) #kernel_regularizer=l2(0.1) for L2 regularization
    model.add(Dense(256, activation='relu')) #kernel_regularizer=l2(0.1) for L2 regularization
    model.add(Dense(y.shape[1], activation='sigmoid'))

    opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.6, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', 'mse'])

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
    mc = ModelCheckpoint('CNN_1D_all.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max') #saving the model with the best accuracy

    history = model.fit(X[train], y[train], batch_size=256, epochs=200, callbacks= [es, mc], validation_data = (X[test], y[test]))

    scores = model.evaluate(X[test], y[test])

    loss_per_fold.append(scores[0])
    acc_per_fold.append(scores[1])
    mse_per_fold.append(scores[2])

print(f'Average loss:  {np.mean(loss_per_fold)}, Average accuracy: {np.mean(acc_per_fold)}, Average MSE: {np.mean(mse_per_fold)}')

# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model scores')
plt.ylabel('scores')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy', 'loss', 'val_loss', 'mse', 'val_mse'], loc='lower left')
plt.show()

# Predictions

In [ ]:
model.load_weights('CNN_1D_all.hdf5')

In [ ]:
predictions = model.predict(X_final, verbose = 1)
predictions

125/125 [==============================] - 1s 8ms/step


array([[0.39459252, 0.32904056, 0.4362493 , ..., 0.34150591, 0.32789463,
        0.32403606],
       [0.3945926 , 0.32904065, 0.4362493 , ..., 0.34150603, 0.3278948 ,
        0.32403618],
       [0.3945926 , 0.32904065, 0.4362493 , ..., 0.34150597, 0.32789475,
        0.32403615],
       ...,
       [0.39459258, 0.3290406 , 0.4362493 , ..., 0.34150591, 0.32789466,
        0.32403606],
       [0.39459267, 0.3290408 , 0.43624938, ..., 0.34150618, 0.3278949 ,
        0.32403633],
       [0.39459267, 0.3290408 , 0.43624938, ..., 0.34150618, 0.3278949 ,
        0.32403633]], dtype=float32)

In [ ]:
predictions = np.round_(predictions)
predictions = predictions.astype(int)
predictions

In [ ]:
scores = model.evaluate(X_final, y_final)

125/125 [==============================] - 2s 16ms/step - loss: 0.5314 - accuracy: 0.2583 - mse: 0.1712
